## CELDA 1: Configuración de Rutas

In [1]:
import sys
import os

# Agregar rutas del proyecto
sys.path.append(os.path.abspath(".."))  # raíz del proyecto
sys.path.append(os.path.abspath("../src"))  # carpeta src

# Verificar rutas
print("Rutas configuradas:")
for path in sys.path[-3:]:
    print(f"  - {path}")

Rutas configuradas:
  - /home/arturoallen/proyecto_52_sistemas/lib/python3.12/site-packages
  - /home/arturoallen/proyecto_52_sistemas/proyecto_sistemas
  - /home/arturoallen/proyecto_52_sistemas/proyecto_sistemas/src


## CELDA 2: Descargar Recursos de NLTK

In [2]:
import nltk

print("Descargando recursos de NLTK...")
nltk.download('punkt_tab', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
print("✓ NLTK configurado")

Descargando recursos de NLTK...
✓ NLTK configurado


## CELDA 3: Inicializar Apache Spark

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ProyectoFinal_MapReduce") \
    .master("local[2]") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.sql.shuffle.partitions", "4") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("WARN")

print("✓ Spark inicializado")
print(f"  Version: {spark.version}")
print(f"  Master: {sc.master}")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/10 22:48:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


✓ Spark inicializado
  Version: 4.0.1
  Master: local[2]


## CELDA 4: Importar Funciones de Utilidad

In [4]:
from src.utils import (
    read_txt,
    strip_gutenberg_headers,
    preprocess_text,
    load_all_books
)

print("✓ Utilidades importadas")

✓ Utilidades importadas


## CELDA 5 Cargar 100 Libros

In [6]:
data_dir = "../data"
books = load_all_books(data_dir, max_books=100)

print(f"\n Resumen de carga:")
print(f"   Total de libros: {len(books)}")
if books:
    ejemplo = books[0]
    print(f"   Ejemplo - ID: {ejemplo[0]}, Tokens: {len(ejemplo[3])}")


📚 Cargando 100 libros desde ../data/
  ✓ Procesados 10/100 libros
  ✓ Procesados 20/100 libros
  ✓ Procesados 30/100 libros
  ✓ Procesados 40/100 libros
  ✓ Procesados 50/100 libros
  ✓ Procesados 60/100 libros
  ✓ Procesados 70/100 libros
  ✓ Procesados 80/100 libros
  ✓ Procesados 90/100 libros
  ✓ Procesados 100/100 libros
✅ Total de libros cargados exitosamente: 100


📊 Resumen de carga:
   Total de libros: 100
   Ejemplo - ID: A Christmas Carol - Charles Dickens, Tokens: 13396


## CELDA 6: Crear RDD de Libros

In [7]:
print("\n" + "="*80)
print("CREANDO RDD DE SPARK")
print("="*80)

# Crear RDD con estructura: (book_id, title, tokens)
books_rdd = sc.parallelize([
    (book[0], book[1], book[3])  # (id, filename, tokens)
    for book in books
])

print(f"✓ RDD creado con {books_rdd.count()} documentos")
print(f"  Particiones: {books_rdd.getNumPartitions()}")



🔥 CREANDO RDD DE SPARK


25/12/10 22:50:10 WARN TaskSetManager: Stage 0 contains a task of very large size (29816 KiB). The maximum recommended task size is 1000 KiB.
[Stage 0:>                                                          (0 + 2) / 2]

✓ RDD creado con 100 documentos
  Particiones: 2


## CELDA 7: CALCULAR TF (Term Frequency) CON MAPREDUCE


In [8]:
print("\n" + "="*80)
print("CALCULAR TF (TERM FREQUENCY) CON MAPREDUCE")
print("="*80)
print("\nFórmula: TF(d,t) = frecuencia de término t en documento d\n")

# MAP: Para cada documento, emitir pares ((doc_id, palabra), 1)
print("🔹 MAP: Emitiendo pares ((doc_id, palabra), 1)...")

tf_pairs = books_rdd.flatMap(
    lambda x: [((x[0], word), 1) for word in x[2]]  # ((book_id, word), 1)
)

print(f"  ✓ {tf_pairs.count():,} pares emitidos")

# REDUCE: Sumar frecuencias por (doc_id, palabra)
print("\n🔹 REDUCE: Sumando frecuencias por (documento, palabra)...")

tf_counts = tf_pairs.reduceByKey(lambda a, b: a + b)

print(f"  ✓ {tf_counts.count():,} términos únicos procesados")




CALCULAR TF (TERM FREQUENCY) CON MAPREDUCE

Fórmula: TF(d,t) = frecuencia de término t en documento d

🔹 MAP: Emitiendo pares ((doc_id, palabra), 1)...


25/12/10 22:50:12 WARN TaskSetManager: Stage 1 contains a task of very large size (29816 KiB). The maximum recommended task size is 1000 KiB.
                                                                                

  ✓ 6,760,238 pares emitidos

🔹 REDUCE: Sumando frecuencias por (documento, palabra)...


25/12/10 22:50:15 WARN TaskSetManager: Stage 2 contains a task of very large size (29816 KiB). The maximum recommended task size is 1000 KiB.
[Stage 2:=============================>                             (1 + 1) / 2]

  ✓ 812,971 términos únicos procesados


## CELDA 8: CALCULAR DF (Document Frequency) CON MAPEO REDUCCION

In [9]:
print("\n" + "="*80)
print("CALCULAR DF (DOCUMENT FREQUENCY) CON MAPREDUCE")
print("="*80)
print("\nFórmula: DF(t) = número de documentos que contienen el término t\n")

# MAP: Emitir (palabra, 1) para cada aparición única en un documento
print("🔹 MAP: Emitiendo pares (palabra, 1)...")

df_pairs = tf_counts.map(
    lambda x: (x[0][1], 1)  # (word, 1) - solo la palabra
)

# REDUCE: Contar en cuántos documentos aparece cada palabra
print("\n🔹 REDUCE: Contando documentos por palabra...")

df_counts = df_pairs.reduceByKey(lambda a, b: a + b)

print(f"  ✓ {df_counts.count():,} palabras únicas en el vocabulario")

'''
# Verificación
ejemplo_df = df_counts.take(5)
print("\n🔍 Ejemplos de DF:")
for word, count in ejemplo_df:
    print(f"  Palabra '{word}' → DF = {count} documentos")
'''



📊 PASO 2: CALCULAR DF (DOCUMENT FREQUENCY) CON MAPREDUCE

Fórmula: DF(t) = número de documentos que contienen el término t

🔹 MAP: Emitiendo pares (palabra, 1)...

🔹 REDUCE: Contando documentos por palabra...


[Stage 5:>                                                          (0 + 2) / 2]

  ✓ 116,707 palabras únicas en el vocabulario


'\n# Verificación\nejemplo_df = df_counts.take(5)\nprint("\n🔍 Ejemplos de DF:")\nfor word, count in ejemplo_df:\n    print(f"  Palabra \'{word}\' → DF = {count} documentos")\n'

## CELDA 9: CALCULAR IDF CON MAP

In [10]:
print("\n" + "="*80)
print("CALCULAR IDF (INVERSE DOCUMENT FREQUENCY) CON MAP")
print("="*80)
print("\nFórmula: IDF(t) = log(N / DF(t))")
print(f"  donde N = {books_rdd.count()} (total de documentos)\n")

import math

N = books_rdd.count()

# MAP: Calcular IDF para cada palabra
print("🔹 MAP: Calculando IDF = log(N / DF) para cada palabra...")

idf_values = df_counts.map(
    lambda x: (x[0], math.log(N / x[1]))  # (word, IDF)
)

print(f"  ✓ IDF calculado para {idf_values.count():,} palabras")
'''
# Verificación
ejemplo_idf = idf_values.take(5)
print("\n Ejemplos de IDF:")
for word, idf in ejemplo_idf:
    df = N / math.exp(idf)  # Recuperar DF
    print(f"  '{word}': DF={int(df)} docs → IDF = log({N}/{int(df)}) = {idf:.4f}")
'''


25/12/10 22:50:26 WARN TaskSetManager: Stage 7 contains a task of very large size (29816 KiB). The maximum recommended task size is 1000 KiB.



📊 PASO 3: CALCULAR IDF (INVERSE DOCUMENT FREQUENCY) CON MAP

Fórmula: IDF(t) = log(N / DF(t))


  donde N = 100 (total de documentos)



25/12/10 22:50:27 WARN TaskSetManager: Stage 8 contains a task of very large size (29816 KiB). The maximum recommended task size is 1000 KiB.
                                                                                

🔹 MAP: Calculando IDF = log(N / DF) para cada palabra...
  ✓ IDF calculado para 116,707 palabras


'\n# Verificación\nejemplo_idf = idf_values.take(5)\nprint("\n🔍 Ejemplos de IDF:")\nfor word, idf in ejemplo_idf:\n    df = N / math.exp(idf)  # Recuperar DF\n    print(f"  \'{word}\': DF={int(df)} docs → IDF = log({N}/{int(df)}) = {idf:.4f}")\n'

## CELDA 10: CALCULAR TF-IDF CON JOIN + MAP

In [11]:
print("\n" + "="*80)
print("CALCULAR TF-IDF = TF × IDF CON JOIN Y MAP")
print("="*80)
print("\nFórmula: TF-IDF(d,t) = TF(d,t) × IDF(t)\n")

# Preparar datos para JOIN
# TF: ((doc_id, word), count) → (word, (doc_id, count))
print("🔹 Preparando datos para JOIN...")

tf_for_join = tf_counts.map(
    lambda x: (x[0][1], (x[0][0], x[1]))  # (word, (doc_id, tf_count))
)

# IDF: (word, idf) - ya está en formato correcto

# JOIN: Combinar TF con IDF por palabra
print("\n🔹 JOIN: Combinando TF con IDF...")

tf_idf_joined = tf_for_join.join(idf_values)
# Resultado: (word, ((doc_id, tf_count), idf))

print(f"  ✓ {tf_idf_joined.count():,} combinaciones TF-IDF")

# MAP: Calcular TF-IDF = TF × IDF
print("\n🔹 MAP: Multiplicando TF × IDF...")

tfidf_scores = tf_idf_joined.map(
    lambda x: ((x[1][0][0], x[0]), x[1][0][1] * x[1][1])
    # ((doc_id, word), tf * idf)
)

print(f"  ✓ {tfidf_scores.count():,} valores TF-IDF calculados")

'''
# Verificación detallada
print("\n🔍 Verificación de cálculo TF-IDF:")
print("="*80)

ejemplo_libro = books[0][0]  # Primer libro
ejemplos_completos = tfidf_scores.filter(
    lambda x: x[0][0] == ejemplo_libro
).take(5)
'''


CALCULAR TF-IDF = TF × IDF CON JOIN Y MAP

Fórmula: TF-IDF(d,t) = TF(d,t) × IDF(t)

🔹 Preparando datos para JOIN...

🔹 JOIN: Combinando TF con IDF...


  ✓ 812,971 combinaciones TF-IDF

🔹 MAP: Multiplicando TF × IDF...


  ✓ 812,971 valores TF-IDF calculados


'\n# Verificación detallada\nprint("\n🔍 Verificación de cálculo TF-IDF:")\nprint("="*80)\n\nejemplo_libro = books[0][0]  # Primer libro\nejemplos_completos = tfidf_scores.filter(\n    lambda x: x[0][0] == ejemplo_libro\n).take(5)\n'

## CELDA 11: NORMALIZAR TF-IDF CON MAPREDUCE

In [12]:
print("\n" + "="*80)
print("NORMALIZAR VECTORES TF-IDF CON MAPREDUCE")
print("="*80)
print("\nFórmula: TF-IDF_norm(d,t) = TF-IDF(d,t) / ||TF-IDF(d)||")
print("  donde ||v|| = √(v₁² + v₂² + ... + vₙ²)\n")

# MAP: Calcular cuadrado de cada TF-IDF → (doc_id, tfidf²)
print("🔹 MAP: Calculando cuadrados de TF-IDF...")

squared_tfidf = tfidf_scores.map(
    lambda x: (x[0][0], x[1] ** 2)  # (doc_id, tfidf²)
)

# REDUCE: Sumar cuadrados por documento → norma²
print("\n🔹 REDUCE: Sumando cuadrados por documento...")

norms_squared = squared_tfidf.reduceByKey(lambda a, b: a + b)

print(f"  ✓ Normas calculadas para {norms_squared.count()} documentos")

# MAP: Calcular raíz cuadrada → norma
print("\n🔹 MAP: Calculando raíces cuadradas (normas)...")

norms = norms_squared.map(
    lambda x: (x[0], math.sqrt(x[1]))  # (doc_id, ||TF-IDF||)
)

'''
# Verificación de normas
ejemplo_normas = norms.take(3)
print("\n🔍 Ejemplos de normas:")
for doc_id, norm in ejemplo_normas:
    print(f"  Documento '{doc_id}' → ||TF-IDF|| = {norm:.2f}")
'''
# JOIN: Combinar TF-IDF con su norma
print("\n🔹 JOIN: Combinando TF-IDF con normas...")

tfidf_with_norm = tfidf_scores.map(
    lambda x: (x[0][0], (x[0][1], x[1]))  # (doc_id, (word, tfidf))
).join(norms)
# Resultado: (doc_id, ((word, tfidf), norm))

# MAP: Normalizar dividiendo por norma
print("\n🔹 MAP: Normalizando (dividiendo por norma)...")

tfidf_normalized = tfidf_with_norm.map(
    lambda x: ((x[0], x[1][0][0]), x[1][0][1] / x[1][1])
    # ((doc_id, word), tfidf_normalized)
)

print(f"  ✓ {tfidf_normalized.count():,} valores normalizados")
print("\nTF-IDF normalizado completado")



NORMALIZAR VECTORES TF-IDF CON MAPREDUCE

Fórmula: TF-IDF_norm(d,t) = TF-IDF(d,t) / ||TF-IDF(d)||
  donde ||v|| = √(v₁² + v₂² + ... + vₙ²)

🔹 MAP: Calculando cuadrados de TF-IDF...

🔹 REDUCE: Sumando cuadrados por documento...


  ✓ Normas calculadas para 100 documentos

🔹 MAP: Calculando raíces cuadradas (normas)...

🔹 JOIN: Combinando TF-IDF con normas...

🔹 MAP: Normalizando (dividiendo por norma)...


[Stage 30:=============================>                            (4 + 2) / 8]

  ✓ 812,971 valores normalizados

✅ TF-IDF normalizado completado


##  CELDA 12: CREAR VOCABULARIO Y MAPEOS

In [13]:
print("\n" + "="*80)
print("CREANDO VOCABULARIO Y MAPEOS")
print("="*80)

# Obtener vocabulario único
print("\n🔹 Extrayendo vocabulario único...")

vocab_rdd = tfidf_normalized.map(lambda x: x[0][1]).distinct().sortBy(lambda x: x)
vocabulary = vocab_rdd.collect()

print(f"  ✓ Vocabulario: {len(vocabulary)} palabras únicas")

# Crear mapeos palabra ↔ índice
word_to_idx = {word: idx for idx, word in enumerate(vocabulary)}
idx_to_word = {idx: word for word, idx in word_to_idx.items()}
'''
print(f"  ✓ Mapeos creados")
print(f"\n🔍 Top 10 palabras del vocabulario:")
for i, word in enumerate(vocabulary[:10], 1):
    print(f"  {i}. '{word}'")
'''
# Crear mapeo de IDs de libros
book_ids = books_rdd.map(lambda x: x[0]).collect()
book_id_to_idx = {bid: idx for idx, bid in enumerate(book_ids)}
idx_to_book_id = {idx: bid for bid, idx in book_id_to_idx.items()}

print(f"\n📖 Total de libros: {len(book_ids)}")



📚 CREANDO VOCABULARIO Y MAPEOS

🔹 Extrayendo vocabulario único...


25/12/10 22:50:46 WARN TaskSetManager: Stage 53 contains a task of very large size (29816 KiB). The maximum recommended task size is 1000 KiB.


  ✓ Vocabulario: 116707 palabras únicas

📖 Total de libros: 100


## CELDA 13: CALCULAR SIMILITUD COSENO CON MAPREDUCE

In [14]:
print("\n" + "="*80)
print(" CALCULAR SIMILITUD COSENO CON MAPREDUCE")
print("="*80)
print("\nFórmula: sim(d₁, d₂) = Σ(TF-IDF₁(t) × TF-IDF₂(t))")
print("  (suma de productos de TF-IDF normalizados)\n")

# Convertir a índices numéricos
print("🔹 Convirtiendo a índices numéricos...")

tfidf_indexed = tfidf_normalized.map(
    lambda x: ((book_id_to_idx[x[0][0]], word_to_idx[x[0][1]]), x[1])
    # ((doc_idx, word_idx), tfidf_norm)
)

# Crear pares de documentos para producto cartesiano
print("\n🔹 Preparando para producto cartesiano...")

# Formato: (word_idx, (doc_idx, tfidf))
tfidf_by_word = tfidf_indexed.map(
    lambda x: (x[0][1], (x[0][0], x[1]))  # (word_idx, (doc_idx, tfidf))
)

print("\n🔹 JOIN: Auto-uniendo por palabra (producto cartesiano)...")

# Self-join: combinar documentos que comparten palabras
word_pairs = tfidf_by_word.join(tfidf_by_word)
# Resultado: (word_idx, ((doc1_idx, tfidf1), (doc2_idx, tfidf2)))

print(f"  ✓ {word_pairs.count():,} pares de (documento, palabra) generados")

# MAP: Calcular producto de TF-IDFs
# IMPORTANTE: Filtrar duplicados y auto-similitudes
print("\n🔹 MAP: Calculando productos de TF-IDF (eliminando duplicados)...")

similarity_contributions = word_pairs.filter(
    # Solo tomar pares donde doc1_idx < doc2_idx (evita duplicados y auto-similitud)
    lambda x: x[1][0][0] < x[1][1][0]
).map(
    lambda x: ((x[1][0][0], x[1][1][0]), x[1][0][1] * x[1][1][1])
    # ((doc1_idx, doc2_idx), tfidf1 * tfidf2)
)

print(f"  ✓ {similarity_contributions.count():,} contribuciones únicas calculadas")

# REDUCE: Sumar contribuciones por par de documentos
print("\n🔹 REDUCE: Sumando similitudes por par de documentos...")

similarity_scores = similarity_contributions.reduceByKey(lambda a, b: a + b)

print(f"  ✓ {similarity_scores.count():,} similitudes calculadas")

'''
# Verificación
print("\n🔍 Top 5 pares más similares:")
top_similar = similarity_scores.takeOrdered(5, key=lambda x: -x[1])
for (idx1, idx2), sim in top_similar:
    bid1 = idx_to_book_id[idx1]
    bid2 = idx_to_book_id[idx2]
    if bid1 != bid2:  # Excluir auto-similitud
        print(f"  '{bid1}' ↔ '{bid2}' → similitud = {sim:.4f}")
'''
print("\n Matriz de similitud calculada con MapReduce")



🎯 CALCULAR SIMILITUD COSENO CON MAPREDUCE

Fórmula: sim(d₁, d₂) = Σ(TF-IDF₁(t) × TF-IDF₂(t))
  (suma de productos de TF-IDF normalizados)

🔹 Convirtiendo a índices numéricos...

🔹 Preparando para producto cartesiano...

🔹 JOIN: Auto-uniendo por palabra (producto cartesiano)...


  ✓ 29,774,579 pares de (documento, palabra) generados

🔹 MAP: Calculando productos de TF-IDF (eliminando duplicados)...


  ✓ 14,480,804 contribuciones únicas calculadas

🔹 REDUCE: Sumando similitudes por par de documentos...


[Stage 75:===================================>                    (10 + 2) / 16]

  ✓ 4,946 similitudes calculadas

✅ Matriz de similitud calculada con MapReduce


## CELDA 14: FUNCIÓN DE RECOMENDACIÓN

In [15]:
print("\n" + "="*80)
print("FUNCIÓN DE RECOMENDACIÓN")
print("="*80)

# Recolectar similitudes como diccionario
print("\n🔹 Recolectando similitudes...")
similarity_dict = similarity_scores.collectAsMap()
print(f"  ✓ {len(similarity_dict):,} similitudes almacenadas")

def recomendar_libros_mapreduce(libro_id, N=5):
    """
    Recomienda N libros similares usando las similitudes calculadas con MapReduce.
    """
    # Convertir ID a índice
    if libro_id not in book_id_to_idx:
        raise ValueError(f" Libro '{libro_id}' no encontrado")
    
    doc_idx = book_id_to_idx[libro_id]
    
    # Buscar todas las similitudes con este documento
    similitudes = []
    for (idx1, idx2), score in similarity_dict.items():
        # Como ahora solo guardamos idx1 < idx2, buscar en ambos lados
        if idx1 == doc_idx:
            similitudes.append((idx_to_book_id[idx2], score))
        elif idx2 == doc_idx:
            similitudes.append((idx_to_book_id[idx1], score))
    
    # Ordenar por similitud (de mayor a menor)
    similitudes.sort(key=lambda x: x[1], reverse=True)
    
    # Eliminar duplicados manteniendo el score más alto
    seen = set()
    similitudes_unicas = []
    for book_id, score in similitudes:
        if book_id not in seen:
            seen.add(book_id)
            similitudes_unicas.append((book_id, score))
    
    return similitudes_unicas[:N]

print("\n✓ Función recomendar_libros_mapreduce() creada")



🎬 CREANDO FUNCIÓN DE RECOMENDACIÓN

🔹 Recolectando similitudes...


  ✓ 4,946 similitudes almacenadas

✓ Función recomendar_libros_mapreduce() creada


## CELDA 15: FUNCIÓN DE PALABRAS IMPORTANTES

In [16]:
print("\n" + "="*80)
print(" CREANDO FUNCIÓN DE PALABRAS IMPORTANTES")
print("="*80)

# Recolectar TF-IDF normalizado
print("\n🔹 Recolectando TF-IDF normalizado...")
tfidf_dict = tfidf_normalized.collectAsMap()
print(f"  ✓ {len(tfidf_dict):,} valores TF-IDF almacenados")

def palabras_importantes_mapreduce(libro_id, M=10):
    """
    Encuentra las M palabras más importantes de un libro.
    """
    if libro_id not in book_ids:
        raise ValueError(f" Libro '{libro_id}' no encontrado")
    
    # Filtrar palabras de este documento
    palabras = [
        (word, score)
        for (doc, word), score in tfidf_dict.items()
        if doc == libro_id
    ]
    
    # Ordenar por score
    palabras.sort(key=lambda x: x[1], reverse=True)
    
    return palabras[:M]

print("\n✓ Función palabras_importantes_mapreduce() creada")



🔤 CREANDO FUNCIÓN DE PALABRAS IMPORTANTES

🔹 Recolectando TF-IDF normalizado...


  ✓ 812,971 valores TF-IDF almacenados

✓ Función palabras_importantes_mapreduce() creada


## CELDA 16: CATÁLOGO COMPLETO

In [17]:
print("\n" + "="*80)
print(" CATÁLOGO COMPLETO DE LIBROS")
print("="*80)
print(f"\nTotal: {len(book_ids)} libros\n")

for i, book_id in enumerate(book_ids, 1):
    filename = books_rdd.filter(lambda x: x[0] == book_id).first()[1]
    print(f"{i:3}. [ID: {book_id}] {filename}")

print("\n Proyecto MapReduce completado")
print("="*80)



📚 CATÁLOGO COMPLETO DE LIBROS

Total: 100 libros



25/12/10 22:51:29 WARN TaskSetManager: Stage 90 contains a task of very large size (29816 KiB). The maximum recommended task size is 1000 KiB.


  1. [ID: A Christmas Carol - Charles Dickens] A Christmas Carol - Charles Dickens.txt


25/12/10 22:51:30 WARN TaskSetManager: Stage 91 contains a task of very large size (29816 KiB). The maximum recommended task size is 1000 KiB.
25/12/10 22:51:30 WARN TaskSetManager: Stage 92 contains a task of very large size (29816 KiB). The maximum recommended task size is 1000 KiB.


  2. [ID: A Dictionary of the Art of Printing - William Savage] A Dictionary of the Art of Printing - William Savage.txt
  3. [ID: A Doll's House - Henrik Ibsen] A Doll's House - Henrik Ibsen.txt


...

 98. [ID: War and Peace - Leo Tolstoy] War and Peace - Leo Tolstoy.txt


25/12/10 22:52:57 WARN TaskSetManager: Stage 237 contains a task of very large size (29072 KiB). The maximum recommended task size is 1000 KiB.


 99. [ID: White Nights and Other Stories - Fyodor Dostoyevsky] White Nights and Other Stories - Fyodor Dostoyevsky.txt


25/12/10 22:52:57 WARN TaskSetManager: Stage 238 contains a task of very large size (29816 KiB). The maximum recommended task size is 1000 KiB.
25/12/10 22:52:58 WARN TaskSetManager: Stage 239 contains a task of very large size (29072 KiB). The maximum recommended task size is 1000 KiB.


100. [ID: Wuthering Heights - Emily Bronte] Wuthering Heights - Emily Bronte.txt

✅ Proyecto MapReduce completado


## CELDA 17: QUITANDO WARNINGS

In [19]:
print("\n" + "="*80)
print("⚡ OPTIMIZANDO ACCESO A DATOS (ELIMINAR WARNINGS)")
print("="*80)

# Crear diccionario local de títulos (evita serializar RDD completo)
print("\n🔹 Creando mapeo de títulos...")
book_titles = {book_id: title for book_id, title, _ in books_rdd.collect()}

print(f"  ✓ {len(book_titles)} títulos mapeados en memoria")
print("\n Los warnings de 29816 KiB deberían desaparecer ahora")


⚡ OPTIMIZANDO ACCESO A DATOS (ELIMINAR WARNINGS)

🔹 Creando mapeo de títulos...


25/12/10 22:52:59 WARN TaskSetManager: Stage 240 contains a task of very large size (29816 KiB). The maximum recommended task size is 1000 KiB.


  ✓ 100 títulos mapeados en memoria

✅ Los warnings de 29816 KiB deberían desaparecer ahora


## CELDA 18: RECOMENDACIONES PERSONALIZADAS

In [20]:
print("\n" + "="*80)
print("RECOMENDAR LIRBRO")
print("="*80)

libro_input = input("\nIngresa el ID del libro o número de la lista: ").strip()

try:
    # Si es un número, buscar por índice
    if libro_input.isdigit():
        idx = int(libro_input) - 1
        if 0 <= idx < len(book_ids):
            libro_seleccionado = book_ids[idx]
        else:
            raise ValueError(f"Número fuera de rango (1-{len(book_ids)})")
    else:
        libro_seleccionado = libro_input
    
    # Pedir cantidad de recomendaciones
    n_recs_input = input("¿Cuántas recomendaciones deseas? (default: 5): ").strip()
    n_recs = int(n_recs_input) if n_recs_input else 5
    
    print(f"\n{'='*80}")
    print(f"📖 BUSCANDO RECOMENDACIONES...")
    print(f"{'='*80}\n")
    
    # CAMBIO AQUÍ: Usar diccionario en lugar de filter
    filename = book_titles[libro_seleccionado]
    print(f"Libro seleccionado: {filename}")
    print(f"ID: {libro_seleccionado}\n")
    
    # Generar recomendaciones
    recomendaciones = recomendar_libros_mapreduce(libro_seleccionado, N=n_recs)
    
    print(f"TOP {n_recs} RECOMENDACIONES:\n")
    print(f"{'#':<4} {'SIMILITUD':<12} {'LIBRO'}")
    print("="*80)
    
    for i, (book_id, score) in enumerate(recomendaciones, 1):
        # CAMBIO AQUÍ: Usar diccionario en lugar de filter
        rec_filename = book_titles[book_id]
        
        # Crear barra visual
        bar_length = int(score * 40)
        bar = "█" * bar_length
        
        print(f"{i:<4} {score:>6.4f} ({score*100:>5.1f}%)  {rec_filename}")
        print(f"     {bar}")
    
    print("\n Recomendaciones generadas con MapReduce")
    
except ValueError as e:
    print(f"\n Error: {e}")
except Exception as e:
    print(f"\n Error inesperado: {e}")



🎬 SISTEMA DE RECOMENDACIONES INTERACTIVO



📖 Ingresa el ID del libro o número de la lista:  39
¿Cuántas recomendaciones deseas? (default: 5):  10



📖 BUSCANDO RECOMENDACIONES...

📚 Libro seleccionado: Meditations - Marcus Aurelius.txt
🆔 ID: Meditations - Marcus Aurelius

🎯 TOP 10 RECOMENDACIONES:

#    SIMILITUD    LIBRO
1    0.5848 ( 58.5%)  The Confessions of St Augustine.txt
     ███████████████████████
2    0.3542 ( 35.4%)  Paradise Lost - John Milton.txt
     ██████████████
3    0.3234 ( 32.3%)  The Divine Comedy - Dante Alighieri.txt
     ████████████
4    0.3096 ( 31.0%)  Complete Works of Shakespeare.txt
     ████████████
5    0.1932 ( 19.3%)  The Lesser Key of Solomon - Goetia.txt
     ███████
6    0.1815 ( 18.1%)  Thus Spake Zarathustra - Friedrich Nietzsche.txt
     ███████
7    0.1070 ( 10.7%)  The Iliad - Homer.txt
     ████
8    0.1065 ( 10.6%)  Romeo and Juliet - William Shakespeare.txt
     ████
9    0.0861 (  8.6%)  The Republic - Plato.txt
     ███
10   0.0825 (  8.2%)  Beyond Good and Evil - Friedrich Nietzsche.txt
     ███

✅ Recomendaciones generadas con MapReduce


## CELDA 21: PALABRAS CARACTERÍSTICAS 

In [22]:
print("\n" + "="*80)
print("PESO DE LAS PALABRAS")
print("="*80)

print("\n Analiza las palabras más importantes de un libro")

libro_input = input("\n📖 Ingresa el ID del libro o número: ").strip()

try:
    # Si es un número, buscar por índice
    if libro_input.isdigit():
        idx = int(libro_input) - 1
        if 0 <= idx < len(book_ids):
            libro_seleccionado = book_ids[idx]
        else:
            raise ValueError(f"Número fuera de rango (1-{len(book_ids)})")
    else:
        libro_seleccionado = libro_input
    
    # Pedir cantidad de palabras
    m_palabras_input = input("¿Cuántas palabras mostrar? (default: 10): ").strip()
    m_palabras = int(m_palabras_input) if m_palabras_input else 10
    
    print(f"\n{'='*80}")
    print(f"ANALIZANDO PALABRAS CARACTERÍSTICAS...")
    print(f"{'='*80}\n")
    
    # Obtener nombre del libro
    filename = books_rdd.filter(lambda x: x[0] == libro_seleccionado).first()[1]
    print(f"Libro: {filename}")
    print(f"ID: {libro_seleccionado}\n")
    
    # Obtener palabras importantes
    palabras = palabras_importantes_mapreduce(libro_seleccionado, M=m_palabras)
    
    if not palabras:
        print("No se encontraron palabras para este libro")
    else:
        print(f"TOP {len(palabras)} PALABRAS MÁS IMPORTANTES:\n")
        print(f"{'#':<4} {'PALABRA':<20} {'TF-IDF':<12} {'IMPORTANCIA'}")
        print("="*80)
        
        max_score = palabras[0][1]  # Score más alto para normalizar barras
        
        for i, (palabra, score) in enumerate(palabras, 1):
            # Barra visual proporcional
            bar_length = int((score / max_score) * 40)
            bar = "█" * bar_length
            
            print(f"{i:<4} {palabra:<20} {score:>8.4f}     {bar}")
        
        print("\nAnálisis de palabras completado")
        print("\nLas palabras con mayor TF-IDF son las más características del libro")
        
except ValueError as e:
    print(f"\n Error: {e}")
except Exception as e:
    print(f"\n Error inesperado: {e}")



PESO DE LAS PALABRAS

 Analiza las palabras más importantes de un libro



📖 Ingresa el ID del libro o número:  100
¿Cuántas palabras mostrar? (default: 10):  11



ANALIZANDO PALABRAS CARACTERÍSTICAS...



25/12/11 00:32:11 WARN TaskSetManager: Stage 243 contains a task of very large size (29816 KiB). The maximum recommended task size is 1000 KiB.
25/12/11 00:32:12 WARN TaskSetManager: Stage 244 contains a task of very large size (29072 KiB). The maximum recommended task size is 1000 KiB.


Libro: Wuthering Heights - Emily Bronte.txt
ID: Wuthering Heights - Emily Bronte

TOP 11 PALABRAS MÁS IMPORTANTES:

#    PALABRA              TF-IDF       IMPORTANCIA
1    heathcliff             0.6685     ████████████████████████████████████████
2    linton                 0.5725     ██████████████████████████████████
3    hareton                0.2558     ███████████████
4    earnshaw               0.1802     ██████████
5    cathy                  0.1773     ██████████
6    catherine              0.1632     █████████
7    hindley                0.0975     █████
8    wuthering              0.0901     █████
9    nelly                  0.0863     █████
10   edgar                  0.0801     ████
11   grange                 0.0646     ███

Análisis de palabras completado

Las palabras con mayor TF-IDF son las más características del libro
